In [0]:
import tensorflow as tf
import numpy as np
import os
import pickle

In [0]:
f=open('dataset.npz','rb')
npzfile = np.load(f)
images=npzfile['arr_0']
labels=npzfile['arr_1']

In [7]:
# Split into training and testing set
split_point = int(len(images)*0.9)
test_img=list(images[:-split_point])
test_labels=list(labels[:-split_point])
train_img=list(images[:split_point])
train_labels=list(labels[:split_point])
train_img=np.array(train_img)
test_img=np.array(test_img)
train_labels=np.array(train_labels)
test_labels=np.array(test_labels)
train_img.shape

(22292, 64, 64, 1)

In [0]:
# Initializing learning parameters
lr = 0.0002
epoch = 100
batch_size = 10
sess = tf.InteractiveSession()

In [0]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 1, 1, 1],strides=[1, 2, 2, 1], padding='VALID')

x = tf.placeholder('float',[None,64,64,1])
y_ = tf.placeholder('float',[None,2])

In [0]:
end = 0
def next_training_batch(batch_size):
    global end
    start = end
    end += batch_size
    if end > len(train_img):
        difference = batch_size - (end - len(train_img))
        return_img = np.concatenate((train_img[end-batch_size:],train_img[:difference]))
        return_labels=np.concatenate((train_labels[end-batch_size:],train_labels[:difference]))
        end = difference
    else:
        return_img=train_img[start:end]
        return_labels = train_labels[start:end]
    return (return_img,return_labels)

In [13]:
# First Layer
W_conv1 = weight_variable([2, 2, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1,64,64,1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)
print(h_conv1.shape)
print(h_pool1.shape)
# Second Layer
W_conv2 = weight_variable([3, 3, 32, 64])
b_conv2 = bias_variable([64])
W_conv2_2 = weight_variable([3, 3, 64, 64])
b_conv2_2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_conv2_2 = tf.nn.relu(conv2d(h_conv2, W_conv2_2) + b_conv2_2)
h_pool2 = max_pool_2x2(h_conv2_2)
print(h_conv2.shape)
print(h_pool2.shape)
#Third Layer 
W_conv3 = weight_variable([3, 3, 64, 128])
b_conv3 = bias_variable([128])

h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)
print(h_conv3.shape)
print(h_pool3.shape)
# Fourth Layer
W_conv4 = weight_variable([5, 5, 128, 64])
b_conv4 = bias_variable([64])

'''h_conv4 = tf.nn.relu(conv2d(h_pool3, W_conv4) + b_conv4)
h_pool4 = max_pool_2x2(h_conv4)
print(h_conv4.shape)
print(h_pool4.shape)
#Fifth Layer
W_conv5 = weight_variable([2, 2, 64, 32])
b_conv5 = bias_variable([32])

h_conv5 = tf.nn.relu(conv2d(h_pool4, W_conv5) + b_conv5)
h_pool5 = max_pool_2x2(h_conv5)
print(h_conv5.shape)
print(h_pool5.shape)'''
# Densely Connected Layer
W_fc1 = weight_variable([8 * 8 * 128, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool3, [-1, 8*8*128])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
print(h_pool2_flat.shape)
print(h_fc1.shape)

(?, 64, 64, 32)
(?, 32, 32, 32)
(?, 32, 32, 64)
(?, 16, 16, 64)
(?, 16, 16, 128)
(?, 8, 8, 128)
(?, 8192)
(?, 1024)


In [14]:
# Dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
print(h_fc1_drop.shape)
# Readout Layer
W_fc2 = weight_variable([1024, 2])
b_fc2 = bias_variable([2])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
print(y_conv.shape)

(?, 1024)
(?, 2)


In [0]:
cross_entropy = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())

In [16]:
for i in range(160000):
    batch = next_training_batch(50)
    if i%500 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

step 0, training accuracy 0.46
step 500, training accuracy 0.74
step 1000, training accuracy 0.68
step 1500, training accuracy 0.6
step 2000, training accuracy 0.62
step 2500, training accuracy 0.6
step 3000, training accuracy 0.74
step 3500, training accuracy 0.68
step 4000, training accuracy 0.74
step 4500, training accuracy 0.68
step 5000, training accuracy 0.84
step 5500, training accuracy 0.74
step 6000, training accuracy 0.74
step 6500, training accuracy 0.78
step 7000, training accuracy 0.78
step 7500, training accuracy 0.84
step 8000, training accuracy 0.68
step 8500, training accuracy 0.86
step 9000, training accuracy 0.86
step 9500, training accuracy 0.94
step 10000, training accuracy 0.96
step 10500, training accuracy 0.92
step 11000, training accuracy 0.9
step 11500, training accuracy 0.96
step 12000, training accuracy 0.94
step 12500, training accuracy 0.94
step 13000, training accuracy 0.96
step 13500, training accuracy 0.96
step 14000, training accuracy 1
step 14500, tra

step 154500, training accuracy 1
step 155000, training accuracy 1
step 155500, training accuracy 1
step 156000, training accuracy 1
step 156500, training accuracy 1
step 157000, training accuracy 1
step 157500, training accuracy 1
step 158000, training accuracy 1
step 158500, training accuracy 1
step 159000, training accuracy 1
step 159500, training accuracy 1


In [17]:
# Test Accuracy
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: test_img, y_: test_labels, keep_prob: 1.0}))

test accuracy 0.999596
